# PAT Conv Finetuning

Trained with Google TPU v2

#Set Up

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title Importing

#Installs
!pip install pyarrow fastparquet

# Packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import tensorflow as tf
import os
from IPython.display import clear_output

# Sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

# Keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
#from keras.layers.embeddings import Embedding
from keras.metrics import AUC

# Tf
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import random

In [ ]:
#@title Random Seeds
import random
## SEEDS

# Hard Code Random Seeds.
r1 = 0
r2 = 1

# Set Random Seed
random.seed(r1)
tf.random.set_seed(r2)

In [ ]:
#@title Connect to TPU
print("TensorFlow version:", tf.__version__)

# Connect to the TPU cluster or fall back to CPU/GPU
try:
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver()  # Tries to connect to the TPU
  tf.config.experimental_connect_to_cluster(resolver)
  tf.tpu.experimental.initialize_tpu_system(resolver)
  strategy = tf.distribute.TPUStrategy(resolver)
  devices = tf.config.list_logical_devices('TPU')
  print('TPU devices:', devices)
except ValueError:
  print("Could not connect to TPU; using CPU/GPU strategy instead.")
  strategy = tf.distribute.get_strategy()

# Example computation using the strategy
with strategy.scope():
  a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

  @tf.function
  def matmul_fn(x, y):
    return tf.matmul(x, y)

  z = strategy.run(matmul_fn, args=(a, b))

print(z)

# Hyperparameters & Settings

In [ ]:
# write where you want to save all your files and retrieve encoder
root = "/content/drive/MyDrive/Extra Curricular /ActigraphyTransformer/A-NEW/PAT Experiments /PAT Conv Finetuning/Models"
encoder_root = "/content/drive/MyDrive/Extra Curricular /ActigraphyTransformer/A-NEW/PAT Experiments /PAT Conv Pretraining/Encoders"
results_root = "/content/drive/MyDrive/Extra Curricular /ActigraphyTransformer/A-NEW/PAT Experiments /PAT Conv Finetuning/Results"

In [ ]:
"""
Please Fill out Parameters Below
"""
## Model size
# eg. ["small", "medium", "large", "huge"]
size = "large"

## Mask ratio
# eg. [.25, .50, .75]
mask_ratio = 0.90

## Smoothing
# eg. [True, False]
smoothing = False

## Loss Function
# eg. [True, False], meaning MSE on only the masked portion or everything in the reconstruction
mse_only_masked = False

In [ ]:
"""
Finetuning Specific Settings
"""

## Finetuning Styles
# Add more if needed

finetuning_styles = ["full", "linear_probe"]

In [ ]:
# Encoder naming
mask_name = int(mask_ratio*100)

encoder_name = f"/conv_encoder_{size}_{mask_name}"

if smoothing == True:
  encoder_name = f"{encoder_name}_smoothed"
else:
  encoder_name = f"{encoder_name}_unsmoothed"

if mse_only_masked == True:
  encoder_name = f"{encoder_name}_mse_only_masked.h5"
else:
  encoder_name = f"{encoder_name}_mse_all.h5"

print(encoder_name)

In [ ]:
# Start of finetuning name
ft_name = f"/conv_AcT_{size}_{mask_name}"

if smoothing == True:
  ft_name = f"{ft_name}_smoothed"
else:
  ft_name = f"{ft_name}_unsmoothed"

if mse_only_masked == True:
  ft_name = f"{ft_name}_mse_only_masked"
else:
  ft_name = f"{ft_name}_mse_all"

print(ft_name)

In [ ]:
encoder_path = encoder_root + encoder_name

# Hyperparameter Additional Info

In [ ]:
"""
Model Size
"""
## Model Size
if size == "small":

  patch_size = 18
  embed_dim = 96
  # encoder
  encoder_num_heads = 6
  encoder_ff_dim = 256
  encoder_num_layers = 1
  encoder_rate = 0.1
  # decoder
  decoder_num_heads = 6
  decoder_ff_dim = 256
  decoder_num_layers = 1
  decoder_rate = 0.1

if size == "medium":

  patch_size = 18
  embed_dim = 96
  # encoder
  encoder_num_heads = 12
  encoder_ff_dim = 256
  encoder_num_layers = 2
  encoder_rate = 0.1
  # decoder
  decoder_num_heads = 12
  decoder_ff_dim = 256
  decoder_num_layers = 1
  decoder_rate = 0.1

if size == "large":

  patch_size = 9
  embed_dim = 96
  # encoder
  encoder_num_heads = 12
  encoder_ff_dim = 256
  encoder_num_layers = 4
  encoder_rate = 0.1
  # decoder
  decoder_num_heads = 12
  decoder_ff_dim = 256
  decoder_num_layers = 1
  decoder_rate = 0.1

if size == "huge":

  patch_size = 5
  embed_dim = 96
  # encoder
  encoder_num_heads = 12
  encoder_ff_dim = 256
  encoder_num_layers = 8
  encoder_rate = 0.1
  # decoder
  decoder_num_heads = 12
  decoder_ff_dim = 256
  decoder_num_layers = 1
  decoder_rate = 0.1

In [ ]:
"""
For Finetuning
"""

## Model Size
if size == "small":

  learning_rate = 0.00001
  early_stopping_patience = 250

  reduce_lr_patience = 75
  min_lr = 1e-6

if size == "medium":

  learning_rate = 0.00001
  early_stopping_patience = 250

  reduce_lr_patience = 75
  min_lr = 1e-6

if size == "large":

  learning_rate = 0.000001
  early_stopping_patience = 250

  reduce_lr_patience = 75
  min_lr = 1e-7


if size == "huge":

  learning_rate = 0.0000005
  early_stopping_patience = 250

  reduce_lr_patience = 100
  min_lr = 1e-8

In [ ]:
"""
Smoothing
"""
if smoothing == True:
  data_folder_location = "/content/drive/MyDrive/Extra Curricular /ActigraphyTransformer/A-NEW/Baseline Tests/Data_2013/All_Meds/Smooth/TestSize2000_set1"

else:
  data_folder_location = "/content/drive/MyDrive/Extra Curricular /ActigraphyTransformer/A-NEW/Baseline Tests/Data_2013/All_Meds/Raw/TestSize2000_set1"

# Process Data

In [ ]:
# Which sizes to look at
train_sizes = [100, 250, 500, 1000, 2500, 5769] # for PAT hyperparameter tuning, we can test on less datasets
test_size = 2000 # fixed

In [ ]:
# first save the test sets
X_test = np.load(os.path.join(data_folder_location, f'X_test_{test_size}.npy'))
y_test = np.load(os.path.join(data_folder_location, f'y_test_{test_size}.npy'))


# Scale the test set
scaler = StandardScaler()
scaler.fit(X_test)
X_test = scaler.transform(X_test)

In [ ]:
print(X_test.shape)
print(y_test.shape)

In [ ]:
train_sets = {}
val_sets = {}

In [ ]:
for size in train_sizes:
  X_train = np.load(os.path.join(data_folder_location, f'X_train_{size}.npy'))
  y_train = np.load(os.path.join(data_folder_location, f'y_train_{size}.npy'))
  train_sets[size] = (X_train, y_train)

  X_val = np.load(os.path.join(data_folder_location, f'X_val_{size}.npy'))
  y_val = np.load(os.path.join(data_folder_location, f'y_val_{size}.npy'))
  val_sets[size] = (X_val, y_val)

print("Data loaded successfully.")
print(f"Train set size: {len(train_sets)}")
print(f"Val set size: {len(val_sets)}")

In [ ]:
for key, value in train_sets.items():
  print(f"For train size {key}: ")

  # print the shapes of X train and y train
  print(f"X train shape: {value[0].shape}")
  print(f"y train shape: {value[1].shape}")

  # also print the shapes of X val and y val
  print(f"X val shape: {val_sets[key][0].shape}")
  print(f"y val shape: {val_sets[key][1].shape}")

  print("================================")

# Wait for later

# LOAD PAT


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, Model

# Modified Transformer Block to output attention weights with explicit layer names (otherwise the same as the )
def TransformerBlock(embed_dim, num_heads, ff_dim, rate=0.1, name_prefix="encoder"):
    input_layer = layers.Input(shape=(None, embed_dim), name=f"{name_prefix}_input")
    attention_layer = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name=f"{name_prefix}_attention")
    attention_output, attention_weights = attention_layer(input_layer, input_layer, return_attention_scores=True)
    attention_output = layers.Dropout(rate, name=f"{name_prefix}_dropout")(attention_output)
    out1 = layers.LayerNormalization(epsilon=1e-6, name=f"{name_prefix}_norm1")(input_layer + attention_output)
    ff_output = layers.Dense(ff_dim, activation="relu", name=f"{name_prefix}_ff1")(out1)
    ff_output = layers.Dense(embed_dim, name=f"{name_prefix}_ff2")(ff_output)
    ff_output = layers.Dropout(rate, name=f"{name_prefix}_dropout2")(ff_output)
    final_output = layers.LayerNormalization(epsilon=1e-6, name=f"{name_prefix}_norm2")(out1 + ff_output)
    return models.Model(inputs=input_layer, outputs=[final_output, attention_weights], name=f"{name_prefix}_transformer")

# Sine/Cosine positional embeddings
def get_positional_embeddings(num_patches, embed_dim):
    position = tf.range(num_patches, dtype=tf.float32)[:, tf.newaxis]
    div_term = tf.exp(tf.range(0, embed_dim, 2, dtype=tf.float32) * (-tf.math.log(10000.0) / embed_dim))
    pos_embeddings = tf.concat([tf.sin(position * div_term), tf.cos(position * div_term)], axis=-1)
    return pos_embeddings


In [ ]:
# Function to load the encoder and build the fine-tuning model with consistent patching and positional embedding
def create_finetuning_model(encoder_path=encoder_path, input_size=10080, patch_size=patch_size, embed_dim=embed_dim, return_attention=False):

    # Load the saved encoder model
    encoder_model = tf.keras.models.load_model(encoder_path, custom_objects={'TransformerBlock': TransformerBlock, 'get_positional_embeddings': get_positional_embeddings})

    # Define new inputs for the fine-tuning model
    inputs = layers.Input(shape=(input_size,), name="finetuning_inputs")

    # Get encoder outputs
    encoder_outputs = encoder_model(inputs)
    encoder_outputs, attention_weights = encoder_outputs[0], encoder_outputs[1:]

    # Pass through a GlobalAveragePooling layer
    x = layers.GlobalAveragePooling1D(name="global_avg_pool")(encoder_outputs)
    x = layers.Dropout(0.1, name="dropout")(x)
    x = layers.Dense(128, activation='relu', name="dense_128")(x)
    outputs = layers.Dense(1, activation="sigmoid", name="output")(x)

    # Include attention weights in the final model outputs if requested
    if return_attention:
        outputs = [outputs] + attention_weights

    # Create and return the fine-tuning model
    finetuning_model = models.Model(inputs=inputs, outputs=outputs, name="finetuning_model")
    return finetuning_model


## Compiling

In [ ]:
# Compile the model -----
with strategy.scope():
  train_model = create_finetuning_model(return_attention=False)
  train_model.compile(
    # Metrics
    loss= tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics= tf.keras.metrics.AUC(name='auc'),
    # Optimizer
    optimizer= tf.keras.optimizers.Adam(
      learning_rate=learning_rate,
      beta_1=0.9,
      beta_2=0.999,
      epsilon=1e-07,
      amsgrad=False
))

# Save the original model weights
train_model.save_weights('original_model_weights.h5')

train_model.summary()

# Training

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# # reduce learning rate (Don't use this it's buggy)
# reduce_lr = ReduceLROnPlateau(
#     monitor='val_loss',    # Monitor validation loss
#     factor=0.5,            # Reduce rate by a factor of 0.5
#     patience=75,           # Number of epochs with no improvement after which learning rate will be reduced
#     min_lr=1e-6,           # Minimum learning rate that the reduction can reach
#     verbose=1              # Print messages when reducing the learning rate
# )

# earlyStopping callback
early_stopper = EarlyStopping(
    monitor='val_auc',  # monitor validation AUC
    mode='max',  # maximize AUC
    patience=early_stopping_patience,  # number of epochs with no improvement after which training will be stopped
    verbose=1,  # display messages when early stopping is triggered
    restore_best_weights=True  # restore model weights from the epoch with the best value of the monitored quantity
)

In [ ]:
# Training Loop

scores = {}
scores["test"] = {}
scores["val"] = {}

for finetuning_style in finetuning_styles:
  print(f"\nFinetuning Style: {finetuning_style}")

  scores["test"][finetuning_style] = {}
  scores["val"][finetuning_style] = {}

  for size in train_sizes:

    print(f"\nSIZE:{size}")


    # Load X_train and fit
    X_train, y_train = train_sets[size]
    train_scalar = StandardScaler()
    train_scalar.fit(X_train)
    X_train = train_scalar.transform(X_train)

    # Load X_val and fit
    X_val, y_val = val_sets[size]
    val_scalar = StandardScaler()
    val_scalar.fit(X_val)
    X_val = val_scalar.transform(X_val)

    # Set Class Weights = Balance
    class1 = sum(y_train)
    total = len(y_train)
    class0 = total-class1

    class_weights = {0: (class1/total),
                  1: ((class0/total))}

    # Reset model weights
    train_model.load_weights('original_model_weights.h5')

    if finetuning_style == "linear_probe":
      for layer in train_model.layers:
        if layer.name == "encoder_model":
            layer.trainable = False

    if finetuning_style == "full":
      for layer in train_model.layers:
        layer.trainable = True

    print(" ")
    # Verify by printing each layer's name and trainable status
    for layer in train_model.layers:
        print(layer.name, layer.trainable)
    print(" ")

    # Train model
    history = train_model.fit(
        X_train, y_train,
        epochs= 10000, # Edit
        batch_size= 64,
        validation_data = (X_val, y_val),
        shuffle=False,
        class_weight=class_weights,
        callbacks = [early_stopper],
        verbose = 2)

    # Save model
    current_model_name = f"{ft_name}_n{size}_{finetuning_style}.h5"
    print(current_model_name)
    train_model.save(root+current_model_name)

    # Test model
    test_scores = train_model.evaluate(X_test, y_test, batch_size=64) # Test Set
    scores["test"][finetuning_style][size] = test_scores[1]
    print("Test AUC:", test_scores[1])

    val_scores = train_model.evaluate(X_val, y_val, batch_size=64) # Val Set

    scores["val"][finetuning_style][size] = val_scores[1]
    print("Val AUC:", val_scores[1])

# Save all results in a .txt
print("\n\n")
print(scores)

results_path = f"{results_root}{ft_name}_RESULTS.txt"

try:
    file_to_write = open(results_path, 'wt')
    file_to_write.write(str(scores))
    file_to_write.close()

except:
    print("Unable to write to file")